In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf


from tensorflow import keras
from keras import layers
from keras.models import Sequential

import pathlib


In [ ]:
data_dir = "/content/drive/MyDrive/tyres/"
data_dir = pathlib.Path(data_dir).with_suffix('')

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
good = list(data_dir.glob('good/*'))
PIL.Image.open(str(good[0]))

Now the loading process


In [ ]:
batch_size = 32
img_height = 512
img_width = 512

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

Just to visualize data

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(12):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(800).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
num_classes = len(class_names)

model = Sequential([
  tf.keras.Input(shape=(img_height, img_width, 3)),
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


Now we ADD dropou layer and a data_augmentation layer since we noted a bit of overfittng

In [ ]:
data_augmentation = keras.Sequential(
  [

    layers.RandomFlip("horizontal"),
                      #input_shape=(img_height,
                       #           img_width,
                        #          3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)


In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")


In [ ]:
num_classes = len(class_names)

model = Sequential([

  tf.keras.Input(shape=(img_height, img_width, 3)),
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2), 
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

#callback functions



callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, min_lr=1e-5),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
]

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.summary()


Now we train the model- using 15 epochs and loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
epochs = 16
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
  callbacks=callbacks  
)

Let's visualize the training results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


# Now we test the model on tyres never been seen

In [ ]:
#create a test dataset

test_ds = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/folder/new_data_tyres/',
    image_size=(img_height, img_width),  
    batch_size=batch_size,               
    shuffle=False                        
)

normalization_layer = tf.keras.layers.Rescaling(1./255)

# Precision, Recall and F1-Score param calculator

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score

# Assuming 'test_ds' is your test dataset
y_pred = model.predict(test_ds)

# Convert the predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)


# Assuming 'test_ds' is a tf.data.Dataset object
y_true_labels = np.concatenate([y for x, y in test_ds], axis=0)


# Calculate precision and recall
precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
f1_score= f1_score(y_true_labels, y_pred_labels , average='weighted')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1_score: {f1_score:.2f}')


# Confusion Matrix Plotting

In [ ]:
y_pred = model.predict(test_ds)
y_pred_labels = np.argmax(y_pred, axis=1)

y_true_labels = np.concatenate([y for x, y in test_ds], axis=0)
cm = confusion_matrix(y_true_labels, y_pred_labels)

#normalize
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.show()